<a id="2"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#083069;overflow:hidden"><b>Loan Default Prediction - Data Science Entry Level Test</b></div>

<div style="display:flex;justify-content:center;">
<img src="https://previews.123rf.com/images/turhanyalcin/turhanyalcin1204/turhanyalcin120400004/13323911-stamp-printed-on-the-approved-loan-application-approved.jpg"
style="width:600px;height:600px;" />
</div>

<a id="1.1"></a>
<h2 style="font-family: Georgia; font-size: 25px; font-style: normal; font-weight: normal; text-align:center; display:fill; text-decoration: none; padding:10px; letter-spacing: 2px; color: white; background-color:black;"><b>Challenge Objectives</h2>

<div style="font-family:Georgia;background-color:black; padding:30px; font-size:17px">

💡 In this project we will do binary classififcation on Loan Financial Data (Top Coder Challenge)<br>

💡The main aim of this project is use machine learning to create a model that predicts the likelihood of a customer defaulting on a loan using historical financial data.<br>
</div>

<a id="1.1"></a>
<h2 style="font-family: Georgia; font-size: 25px; font-style: normal; font-weight: normal; text-align:center; display:fill; text-decoration: none; padding:10px; letter-spacing: 2px; color: white; background-color:black;"><b>Importing Libraries</h2>

In [2]:
import pandas as pd
pd.set_option('display.max_columns',100)

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette("muted")
pd.set_option("display.max.rows",None)
pd.set_option("display.max.columns",None)

import warnings
warnings.filterwarnings('ignore')

from scipy import stats as st
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

<a id="1.1"></a>
<h2 style="font-family: Georgia; font-size: 25px; font-style: normal; font-weight: normal; text-align:center; display:fill; text-decoration: none; padding:10px; letter-spacing: 2px; color: white; background-color:black;"><b>Loading Datasets</h2>

In [4]:
data_df = pd.read_csv('Financial_data.csv')
data_df.head(10)

,Customer ID,Annual income,Credit score,Employment length,Debt-to-income ratio,Loan default
0,1,"$65,000",680,2 years,0.24,0.0
1,2,"$40,000",590,1 year,0.37,1.0
2,3,"$95,000",720,5 years,0.16,0.0
3,4,"$35,000",530,6 months,0.50,1.0
4,5,"$80,000",710,4 years,0.27,0.0
5,6,"$55,000",650,3 years,0.32,0.0
6,7,"$100,000",750,6 years,0.15,0.0
7,8,"$45,000",600,1 year,0.42,1.0
8,9,"$110,000",790,7 years,0.12,0.0
9,10,"$75,000",700,3 years,0.28,0.0


<a id="1.1"></a>
<h2 style="font-family: Georgia; font-size: 25px; font-style: normal; font-weight: normal; text-align:center; display:fill; text-decoration: none; padding:10px; letter-spacing: 2px; color: white; background-color:black;"><b>Data Fields </h2>

> * **Customer ID:** This feature indicates the Id of each customer.
> * **Annual income:** The self-reported annual income provided by the customer during registration. 
> * **Credit score:** This is  a prediction of your credit behavior, such as how likely you are to pay a loan back on time, based on information from your credit reports.
> * **Employment length:** This feature indicates employment length in years and months.
> * **Debt-to-income-ratio:** Asuuming this is a ratio calculated using the customers’s total debt, divided by the customers’s self-reported income.
> * **Loan default:**  binary indicator of whether the customer defaulted. **0 means Not- Defaulted** & **1 means Defaulted**


In [5]:
# To check the rows and columns of the dataframe
data_df.shape

(58, 6)

In [8]:
# To check data type and information of each column
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer ID           58 non-null     int64  
 1   Annual income         57 non-null     object 
 2   Credit score          58 non-null     int64  
 3   Employment length     57 non-null     object 
 4   Debt-to-income ratio  57 non-null     float64
 5   Loan default          57 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.8+ KB


In [9]:
# The Annual income column datatype states object, would like to change to integer
# using vectorised str methods to slice each string entry by removing the 's' in
data_df['Annual income'] = data_df['Annual income'].str.replace('$','').replace(',','').astype(int)

ValueError: invalid literal for int() with base 10: '65,000'

In [7]:
data_df.describe()

,Customer ID,Credit score,Debt-to-income ratio,Loan default
count,58.000000,58.000000,57.000000,57.000000
mean,29.500000,674.310345,0.296667,0.298246
std,16.886879,80.461652,0.116792,0.461555
min,1.000000,520.000000,0.110000,0.000000
25%,15.250000,610.000000,0.200000,0.000000
50%,29.500000,695.000000,0.280000,0.000000
75%,43.750000,730.000000,0.380000,1.000000
max,58.000000,800.000000,0.520000,1.000000
